In [1]:
import pandas as pd
from pandas.io import sql

import sqlite3 

In [2]:
conn = sqlite3.connect('dat-test.db')

In [3]:
Client_table = pd.read_csv('Client_table.csv', low_memory=False)
Client_table.head()

,Userid,Payment,Month
0,100,220,4
1,300,300,10
2,400,1000,11
3,100,500,10
4,300,600,11


In [4]:
Fico_Score= pd.read_csv('Fico_Score.csv', low_memory=False)
Fico_Score.head()

,Userid,Fico,Month
0,100,500,4
1,300,600,10
2,400,800,11
3,100,521,10
4,300,660,11


In [5]:
import os
os.getcwd()

'C:\\Users\\jpsabini\\GA_DAT_NYC_39_JULY\\DAT-NYC-39\\lessons\\lesson-15\\code\\starter-code'

### to_sql

In [7]:
Client_table.to_sql('Client_table', if_exists='replace', index=False, con=conn)
Fico_Score.to_sql('Fico_Score',if_exists='replace', index=False,con=conn)

pd.read_sql("""SELECT *  FROM Client_table """, con=conn)


,Userid,Payment,Month
0,100,220,4
1,300,300,10
2,400,1000,11
3,100,500,10
4,300,600,11
5,700,300,1


In [8]:
pd.read_sql(
"""
SELECT
Userid, Fico, Month
FROM Fico_Score
""", con=conn)

,Userid,Fico,Month
0,100,500,4
1,300,600,10
2,400,800,11
3,100,521,10
4,300,660,11


### Inner join with explicit join on /Even if we were to eliminate the select a,b,c,d,e,f the database would join everything.

In [9]:
pd.read_sql('''SELECT c.Userid, c.Payment ,c.Month,f.Userid,f.Month,f.Fico from Client_table 
as c  INNER Join
Fico_Score as f  ON c.Userid=f.userid and c.month = f.month 
Order by c.Userid ;''',con=conn)

,Userid,Payment,Month,Userid,Month,Fico
0,100,220,4,100,4,500
1,100,500,10,100,10,521
2,300,300,10,300,10,600
3,300,600,11,300,11,660
4,400,1000,11,400,11,800


An inner join . Anytime a join is done the joined table is produced first. This is why you have to be careful about joins . They can be expensive.


In [ ]:
### Total_PAyment

In [10]:
pd.read_sql('''SELECT c.Userid, sum(c.Payment)TOTAL_PAYMENT ,AVG(f.Fico) FICO from Client_table 
as c  INNER Join
Fico_Score as f  ON c.Userid=f.userid and c.month = f.month  Group By(c.Userid)
Order by c.Userid ;''',con=conn)

,Userid,TOTAL_PAYMENT,FICO
0,100,720,510.5
1,300,900,630.0
2,400,1000,800.0


In [ ]:
#Using FICO as a conditional

In [102]:
pd.read_sql('''SELECT c.Userid, sum(c.Payment)TOTAL_PAYMENT ,AVG(f.Fico) FICO from Client_table 
as c  INNER Join
Fico_Score as f  ON c.Userid=f.userid and c.month = f.month  Where FICO > 550 Group By(c.Userid) 
Order by c.Userid ;''',con=conn)

,Userid,TOTAL_PAYMENT,FICO
0,300,900,630.0
1,400,1000,800.0


# Note: A left Outer Join. The LEFT TABLE is Client Table SO We Generat all of the entries from the CLient Table and then Join it with the Fico Score. Note the Nan

In [103]:
pd.read_sql('''SELECT c.Userid, sum(c.Payment)TOTAL_PAYMENT ,AVG(f.Fico) FICO from Client_table 
as c   Left Outer Join
Fico_Score as f  ON c.Userid=f.userid and c.month = f.month Group By(c.Userid) 
Order by c.Userid ;''',con=conn)

,Userid,TOTAL_PAYMENT,FICO
0,100,720,510.5
1,300,900,630.0
2,400,1000,800.0
3,700,300,NaN


## We name the selected columns. Total_Payment and FICO we then selct those which have no NAN . 


In [104]:
pd.read_sql('''SELECT c.Userid, sum(c.Payment)TOTAL_PAYMENT ,AVG(f.Fico) FICO from Client_table 
as c   Left Outer Join
Fico_Score as f  ON c.Userid=f.userid and c.month = f.month Where f.fico is not NULL Group By(c.Userid) 
Order by c.Userid ;''',con=conn)

,Userid,TOTAL_PAYMENT,FICO
0,100,720,510.5
1,300,900,630.0
2,400,1000,800.0


In [12]:
pd.read_sql('''
SELECT c.Userid,c.month,f.Fico FICO from Client_table 
as c   Left Outer Join
Fico_Score as f  ON c.Userid=f.userid and c.month = f.month 

UNION 
SELECT f.Userid,f.Month,f.Fico from Client_table 
as c  INNER Join
Fico_Score as f  ON c.Userid=f.userid and c.month = f.month; ''',con=conn)

,c.Userid,c.month,FICO
0,100,4,500.0
1,100,10,521.0
2,300,10,600.0
3,300,11,660.0
4,400,11,800.0
5,700,1,NaN


In [ ]:
##HAPPY THANKSGIVING to youall.